In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.3.9")

In [2]:
import kotlinx.coroutines.*

In [3]:
val POPCOUNT = 10000

fun <A, B> Iterable<A>.pmap(f: suspend (A) -> B): List<B> = runBlocking {
    map { async { f(it) } }.awaitAll()
}

In [4]:
%use lets-plot

In [45]:
val rand = kotlin.random.Random(1)

fun pdf(x: Double) = exp(-x * x / 2) / sqrt(2 * PI)

fun pdf(x: Double, μ: Double, σ: Double) = pdf((x - μ) / σ) / σ

tailrec fun cdf(z: Double, sum: Double = 0.0, term: Double = z, i: Int = 3): Double =
    when {
        z < -8.0 -> 0.0
        z > 8.0 -> 1.0
        sum + term == sum -> 0.5 + sum * pdf(z)
        else -> cdf(z, sum + term, term * z * z / i, i + 2)
    }

fun cdf(z: Double, μ: Double, σ: Double) = cdf((z - μ) / σ)

fun inverseCDF(y: Double, μ: Double = 0.0, σ: Double = 1.0) =
    inverseCDF(y, -8.0, 8.0, μ, σ)

val precision = 0.00000001
tailrec fun inverseCDF(
    y: Double, lo: Double, hi: Double,
    μ: Double, σ: Double,
    mid: Double = lo + (hi - lo) / 2
): Double = when {
    hi - lo < precision -> mid
    cdf(mid, μ, σ) > y -> inverseCDF(y, lo, mid, μ, σ)
    else -> inverseCDF(y, mid, hi, μ, σ)
}

In [47]:
fun compare(vararg samplers: (Double) -> Double) =
    compare(*samplers.map { f -> 
            (1..POPCOUNT).pmap { f(rand.nextDouble()) } 
        }.toTypedArray()
    )

fun compare(vararg samples: List<Double>) =
    lets_plot(mapOf<String, Any>(
        "x" to samples.fold(listOf<Double>()) { acc, function ->
            acc + function
        },
        "" to samples.mapIndexed { i, s -> List(s.size) { "PDF$i" } }.flatten()
    )).let {  
        it + geom_density(alpha = .3) { x = "x"; fill = "" } + ggsize(500, 250)
    }

Suppose we have two normal distributions.

In [50]:
val prA = { it: Double -> inverseCDF(it) }
val prB = { it: Double -> prA(it) * 1.4 + 3.5 }

compare(prA, prB)

How can we combine these distributions? One way is to simply average them.

In [55]:
val prC = { it: Double -> (prA(it) + prB(it)) / 2.0 }
compare(prA, prB, prC)

Another way is to mix them somehow.

In [66]:
val prD = { it: Double -> 
    if(rand.nextBoolean()) prA(it) else prB(it) 
}

compare(prA, prB, prD)